<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/dev/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pollinator

Does the heavy-lifting for **Pollinations.AI** on Google Colab's free GPUs.

Click *Runtime - Run All* and switch back to the **Pollinations** tab of the browser.

![](https://pollinations.ai/Runtime-RunAll.png?)

In [ ]:
#@title Config and Requirements
!pip install coolname gdown papermill json5
from IPython.display import clear_output
from coolname import generate_slug
from os.path import join, basename
from shlex import quote

from glob import glob
from json5 import loads

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

clear_output()
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
ipfs_root = "/content/ipfs"
worker_root = "/content"
notebook_path = join(ipfs_root,"notebook.ipynb")

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}")

     |████████████████████████████████| 143kB 12.0MB/s 
     |████████████████████████████████| 112kB 11.0MB/s eta 0:00:01

In [ ]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!mkdir -p $ipfs_root/status
!mkdir -p $worker_root


!echo "Created IPFS resource" > $ipfs_root/status/log
!echo -n '"initializing"' > $ipfs_root/status/colabState

clear_output()

In [ ]:
#@title Install Pollinations CLI

%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev 
!git pull
!npm run install_backend
%cd -

clear_output()

In [ ]:
#@title Run IPFS daemon &

!/content/pollinations/ipfs/ipfsd_install_colab.sh
!nohup  /content/pollinations/ipfs/ipfsd_forever.sh >> $ipfs_root/status/log_ipfsd &
!sleep 5
clear_output()


node_id=!ipfs id -f="<id>\n"
node_id=node_id[0]

print("---Node ID---:", node_id)

In [ ]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))




In [ ]:
#@title Receive first IPFS state. Watch and send ipfs output folder &

def get_params():
  parameters={}
  for path in glob(f"{ipfs_root}/input/*"):
    key = basename(path)
    with open(path,'r') as f: 
      value = f.read()
      print("reading parameter", value)
      try:
        parameters[key] = loads(value)
      except:
        parameters[key] = ""
  return parameters

def get_form_action():
  params = get_params()
  return "formAction" in params and params["formAction"]
  

def wait_submitted(skip_pollinate=False):
  print("Waiting for form submission")
  #!echo -n '"waiting"' > $ipfs_root/status/colabState
  while not get_form_action() == "submit":
    if skip_pollinate:
      !sleep 2
    else:
      !DEBUG=* pollinate -p $ipfs_root --ipns --receive --once
  !rm -v $ipfs_root/input/formAction
  
wait_submitted()

!python /content/pollinations/pollinations/prepare_for_papermill.py $notebook_path

!nohup pollinate -p $ipfs_root --ipns &

clear_output()

In [ ]:
#@title Notebook Runner

output_path = ipfs_root+"/output"

first_run=True


def run(parameters):
  print("Running",parameters)
 
  parameters["output_path"] = output_path
  _params = [["-p", quote(key), quote(str(value))] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]


  cmd = ["papermill", notebook_path, f"/content/notebook_output.ipynb"] + params + ["--log-output"]
  print("Running cmd"," ".join(cmd))

  with open(ipfs_root+"/status/log","a") as logout:
    !echo -n '"running"' > $ipfs_root/status/colabState
    p = Popen(cmd, 
            stdout = logout, 
            stderr = logout, 
            shell = False,
            text = True,
            cwd = worker_root
            )#.communicate()# as p:
    print("popen",p)
    
    while p.returncode is None:
      form_action = get_form_action()
      print("form action", form_action)
      if form_action == "cancel":
        print("killing!!!")
        !rm -v $ipfs_root/input/formAction
        p.kill()
        break
      if p.returncode is not None:
        print("got return code", p.returncode)
        break
      !sleep 2
  
while True:
  if not first_run:
    wait_submitted(skip_pollinate=True)
    !python /content/pollinations/pollinations/prepare_for_papermill.py $notebook_path
  first_run = False
  parameters = get_params()
  print("New parameters", parameters)
  !echo -n '"running"' > $ipfs_root/status/colabState
  run(parameters)
  !echo -n '"waiting"' > $ipfs_root/status/colabState
